Régression non linéaire
===================

L'objectif de ce TP est d'effectuer les différents calculs des notions vues en cours sur le modèle
$$Y = f(X) + \varepsilon$$
===========================

On commence par importer les librairies dont on aura besoin pour le TP, à savoir *numpy* pour les outils mathématiques et *sklearn* pour les outils d'apprentissage.

Pour que les *plot* s'affiche dans la page, on ajoute

    %matplotlib inline

In [ ]:
%matplotlib inline
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
import matplotlib.pyplot as plt

 ## Méthode des moindres carrés ordinaire, avec une base orthnormée
 
 On se place dans le contexte de régression sur une base orthonormée de $L_2([0;1])$ et dans le cas où le fonction $f$ est dans  
 $$ H(\beta,L)=\{f\in L^2([0;1]) | f^{(\beta-1)} <<\lambda, \int f^{(\beta)}d\lambda\leq L\}.$$
 
 On pose pour tout $k\in\mathbb{N}$, et tout $x\in [0;1]$

 \begin{align}
 \phi_1&=1\\
 \phi_{2k}(x)&=\sqrt{2}\cos(2\pi k x)\\
 \phi_{2k+1}(x)&=\sqrt{2}\sin(2\pi k x)\\
 \end{align}
 
 **Compléter le définition de phi suivante**

In [ ]:
def is_odd(k):
    if k==2*np.trunc(k/2):
        return False
    else:
        return True

def phi(k,x):
    if x<0 or x>1:
        return 0
    elif k==1:
        return 1
    elif is_odd(k):
        return  
    return 0

**De la même façon, définir les fonctions 
$$f_1:x\mapsto \mathbf{1}_{\{0\leq x\leq 1/2\}} 2*x + \mathbf{1}_{\{1/2<x\leq 1\}}\log(x)$$
et 
$$f_2:x\mapsto \mathbf{1}_{\{0\leq x \leq 1\}} \left(\frac{1}{2} - \left|x-\frac{1}{2}\right|\right).$$
**

In [ ]:
def fun(x):

    return x

def fdeux(x):

    return x

On trace le graphe de ces deux fonctions.

In [ ]:
p=1000
fun_plot=[]
for x in np.arange(0,1,1.0/p):
    fun_plot.append(fun(x))
plt.plot(np.arange(0,1,1.0/p),fun_plot,'b.')
fdeux_plot=[]
for x in np.arange(0,1,1.0/p):
    fdeux_plot.append(fdeux(x))
plt.plot(np.arange(0,1,1.0/p),fdeux_plot, 'g--')

On génère alors nos données.

In [ ]:
n=100
eps=0.1*np.random.normal(0,1,n)
Y=np.zeros(n,dtype=float)
for x in range(n):
    Y[x]=fun(float(x)/n)+eps[x]

fig, ax = plt.subplots()
print(Y.shape)
print(np.arange(0,1,1.0/n).shape)
ax.scatter(np.arange(0,1,1.0/n),Y, alpha=0.5)

ax.set_xlabel(r'$X$', fontsize=20)
ax.set_ylabel(r'$Y$', fontsize=20)
ax.set_title(r'$Y$ en fonction de $X$')

ax.grid(True)
fig.tight_layout()

plt.show()

On peut maintenant générer les $\phi_j(x_i)$ sur lesquels on appliquera la régression LASSO.



In [ ]:
m=n-1
phiX=np.zeros((n,m),dtype=np.float)
for x in range(n):
    for j in range(m):
        phiX[x,j]=phi(j+1,float(x)/n)
        
print(phiX)

**Utiliser la méthode LASSO pour déterminer un estimateur de $f$, avec $m=n-1$, pour $f_1$ et $f_2$.**
**Faire varier le coefficient $\lambda$.**

In [ ]:
l=0.2*np.sqrt(np.log(2*m)/n)
print l


**Tracer sur un même graphe, $f_1$ et son estimateur $\hat f_1$. De même pour $f_2$ et son estimateur $\hat f_2$.**

**Calculer l'erreur de l'estimateur: $\frac{1}{n}\sum_{i=1}^n|\hat f(x_i) - f(x_i)|^2$ pour chacune des fonctions**.

**Tracer cette erreur en fonction de $n$ pour $n=10..1000$.**

On cherche maintenant à utiliser l'estimateur des $k$ plus proches voisins.
Pour celà on pourra utiliser

    KNeighborsRegressor(n_neighbors=k)
    neigh.fit(X, Y)
    
**Estimer $f_1$ et $f_2$ à l'aide des $k$-plus proches voisins.
Afficher le graphe des fonctions et leur estimateur.**

In [ ]:
k=int(np.floor(np.power(n,2.0/3.0))+1)
X=np.arange(0,1,1.0/n)
X=X.reshape(-1,1)
print k
neigh = KNeighborsRegressor(n_neighbors=k)
#à partir du cours: alpha=1, d=1 donc k=n^{2/3}
neigh.fit(X, Y)

hatf=[]
Xplot=np.arange(0,1,1.0/p)
Xplot=Xplot.reshape(-1,1)
for x in Xplot:
    hatf.append(neigh.predict([x]))
    
fig, ax = plt.subplots()

ax.scatter(np.arange(0,1,1.0/p),hatf, alpha=0.5)
ax.scatter(np.arange(0,1,1.0/p),fun_plot, alpha=0.5, color='red')

ax.set_xlabel(r'$X$', fontsize=20)
ax.set_ylabel(r'$Y$', fontsize=20)
ax.set_title(r'$Y$ en fonction de $X$')

ax.grid(True)
fig.tight_layout()

plt.show()


On introduit les estimateurs à noyaux.
L'estimateur de Nadaraya-Watson pour un noyau $K$ est donnée par
$$ \hat f(x)=\sum_{i=1}^n w_{i,n}(x)Y_i.$$
où
$$ w_{i,n}(x)=\frac{K\left(\frac{x-X_i}{h}\right)}{\sum_{i=1}^n K\left(\frac{x-X_i}{h}\right)}.$$

$K$ permet de mettre un poids sur chaque $Y_i$ qui dépend de la proximité de $x$ à $X_i$, qui est paramétré par $h>0$.

On peut montrer que si $R=\int K^2(u)du<\infty$ et $K_2=\int K(u)u^2du<\infty$ alors
$$EQM(\hat f(x))\lesssim h^4 + \frac{1}{nh},$$
si la densité de $X$ est minorée et $f$ est deux fois différentiable.

**Calculer la vitesse de convergence de l'EQM pour un choix de $h$ optimal.**

*Réponse:*

On choisit
$$K:u\mapsto e^{-\frac{u^2}{2}}.$$

**Utiliser ce noyau pour calculer l'estimateur de Nadaray-Watson de $f_1$ et $f_2$.**